## Evaluate

$ 5x^{3} - 4x^{2}y^{2} + 13xy^{2} + x^{2} -10y = out $

$ v_1 = xx $

$ v_2 = yy $

$ v_3 = 5xv_1 $

$ v_4 = 4v_1v_2 $

$ out -v_3 + v_4 - v_1 + 10y = 13xv_2 $

$ w = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4
\end{bmatrix} $



$ L = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 5 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 4 & 0 & 0 & 0 \\
0 & 0 & 13 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix} $


$ R = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
\end{bmatrix} $


$ O = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 10 & -1 & 0 & -1 & 1 \\
\end{bmatrix} $


## R1CS

In [1]:
import galois
import numpy as np

p = 71
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w =", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)


w = [  1 104   2   3   4   9  40 144]
Lw = [2 3 4 9 9]
Rw = [ 2  3 10 16 26]
Lw * Rw = [  4   9  40 144 234]
Ow =      [  4   9  40 144 234]


## QAP

In [2]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

In [3]:
T = galois.Poly([1, p-1], field=FP)
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly([1, p-i], field=FP)

In [4]:
import groth16

qap = groth16.QAP(Lp, Rp, Op, T)
pk, vk = groth16.keygen(qap)

print(pk)

print(vk)


----- Prover Key -----
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)]
[τ]G2 = [((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 408236787586343368133220340314543556831685132759340120810574107621412009

In [5]:
proof = groth16.prove(pk, w[:2], w[2:], qap)

print(proof)


----- Proof -----
A = (14614942786781395235323739941473784359708158727987721451462286880826505164913, 21619883166399935899647001848432357053098942563907854290524692662449417028110)
B = ((319587749637165176148886793271196602371225601986791290743038843027662975010, 3799125525057959973194726334738911206921416163026998466480194981504341041059), (14804322218212369892528022736659815952157535301541362469519071425927767448738, 15320077395654528648820749046262517324306647091379407731658843025099270151841))
C = (17701753435269740403921105970374551089935320911024962898677727136701795584514, 14346862312910190547529197118974415823038018600073367279111060787975692996852)
        


In [6]:
groth16.create_verifier(vk, w[:2], proof)